# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [61]:
# general + data loading
import pandas as pd
from sqlalchemy import create_engine

# pre-processing
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re

# machine learning
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

# storage
import pickle
import time

In [36]:
# download relevant wordnet models
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [37]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_data.db')
df = pd.read_sql_table('texts', engine)
df.head()
X = df[['message', 'original', 'genre']]
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [56]:
# init these globally (to avoid call for every run)
stops = set(stopwords.words('english'))
wlem = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    '''Matches the given treebank POS tag to a wordnet one to be used by lemmatizer.'''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

def tokenize(text):
    '''Tokenizes the words and prepares for training.'''
    # remove punctuation
    text = re.sub("[\.\\:;!?'\"-]", " ", text.lower())
    tokens = word_tokenize(text)
    
    # remove stopwords
    tokens = [tok for tok in tokens if tok not in stops]
    
    # part of speech
    tags = [wlem.lemmatize(w, pos=get_wordnet_pos(t)) for w, t in pos_tag(tokens)]
    
    # TODO: further processing (e.g. NER)
    return tags
    
X['message'].head().apply(tokenize)

0    [weather, update, cold, front, cuba, could, pa...
1                                          [hurricane]
2                                [look, someone, name]
3    [un, report, leogane, 80, 90, destroyed, hospi...
4    [say, west, side, haiti, ,, rest, country, tod...
Name: message, dtype: object

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [58]:
# simple baseline model
pipeline = [
    ('vectorize', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=4) )
]
pipeline = Pipeline(pipeline)

Define a function to create model from config file:

In [ ]:
def build_model(config='../models/config.json'):
    '''Creates a new pipeline based on the config information.'''
    # TODO
    pipeline = [
        ('vectorize', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=4) )
    ]
    return pipeline

Define a function to add the test results to the csv file:

In [62]:
def score_and_doc(model, name, X_test, y_test, config='../models/config.json'):
    '''General scoring function.
    
    Takes the model and predicts output against given data.
    Finally scores them along different metrics and writes the results to the experiments file.
    
    Args:
        model: The sklearn model
        name (str): name of the model (used for documentation)
        X_test: test data
        y_test: expected test labels
    '''
    # predict the data
    y_pred = model.predict(X_test)
    
    # score the model
    avg = 'micro'
    f1 = f1_score(y_test, y_pred, average=avg)
    prec = precision_score(y_test, y_pred, average=avg)
    rec = recall_score(y_test, y_pred, average=avg)
    
    # retrieve current config data?
    # TODO
    
    # open file and append
    f=open("../experiments.csv", "a+")
    f.write("{},{:.6f},{:.6f},{:.6f},{}\n".format(name, f1, prec, rec, time.ctime()))
    f.close()
    
    # print output
    print("{}: F1={:.6f} Prec={:.6f} Rec={:.6f}".format(name, f1, prec, rec))

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [59]:
pipeline.fit(X_train['message'], y_train)

Pipeline(memory=None,
         steps=[('vectorize',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function token...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('cls',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1.0,
                                                                    class_weight=None,
                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [63]:
score_and_doc(pipeline, 'baseline', X_test['message'], y_test)

baseline: F1=0.654995 Prec=0.817395 Rec=0.546430


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
score_and_doc(pipeline, 'baseline_v2', X_test['message'], y_test)

0.3671530161844041

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
filename = '../models/pipeline.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.